In [3]:
import pandas as pd

In [7]:
df = pd.read_csv("spotify_millsongdata.csv")

In [5]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [6]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [8]:
df.shape

(57650, 4)

In [9]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [10]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [11]:
df.head(10)

,artist,song,text
0,Lil Wayne,1St Key,I bought my first key from my baby momma broth...
1,Tom T. Hall,Fox Hollow's Animal Train,"When we took off, we almost had a critter of e..."
2,Kenny Chesney,Christmas In Dixie,"By now in New York City, there's snow on the g..."
3,Gary Numan,This Is Emotion,This is \r\nThis is emotion calling \r\nInsi...
4,Unkle,Invasion,Show them only \r\nFlesh you needed \r\nYou ...
5,Isley Brothers,Down Low (Nobody Has To Know),Listen girl you want me but he needs you \r\n...
6,Gordon Lightfoot,Much To My Surprise,I saw two dreamers dreamin' \r\nThey almost c...
7,Lenny Kravitz,God Is Love,God is love \r\nThrough all your trials and t...
8,Etta James,Strung Out,I will do whatever you say \r\nIt's got to be...
9,Opeth,April Ethereal,"It was me, peering through the looking-glass. ..."


In [12]:
df['text'][0]

"I bought my first key from my baby momma brother  \r\nI bought my first key  \r\nBought my bought my first key  \r\nI bought my first key from my baby momma brother  \r\nI bought my first key  \r\nBought my bought my first key  \r\n  \r\nYeah, hustling on my city streets  \r\nTrying to get a whole key (i bought my frist key, my first key)  \r\nIn '93 and we was getting 'em like for twenty five  \r\nColombian connect, homey we was getting fly  \r\nWe on the grind and our nuts got bigga  \r\nAnd every day we in the motherfucking hood our guns got bigga  \r\nStunting on them niggas, getting money staying fly every day a nigga spending  \r\nRiding 25s and we blazing getting high, nigga showing off his ride  \r\nCrusing through the beach nigga rolling in them bimmers  \r\nThem people pull me over hating cause they can't see us  \r\nKnow I got the money (laws won't leave me alone)  \r\nNow they acting funny cause they know a nigga strong  \r\nMoney long and we coming  \r\n  \r\nI bought my 

In [13]:
# df = df.sample(5000)

In [14]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [15]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [31]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [32]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [34]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [35]:
similarity[0]

array([1.        , 0.0121341 , 0.00300424, ..., 0.0479906 , 0.0250803 ,
       0.0223524 ])

In [36]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [37]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

IndexError: index 0 is out of bounds for axis 0 with size 0

In [44]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))